In [1]:
!ls

docker	 maps			    README.md  tools	   utils.py
main.py  original_preprocess.ipynb  run.yaml   torchbeast


In [2]:
import torch
import torch.nn.functional as F
from torchbeast.neural_mmo.train_wrapper import FeatureParser
from torchbeast.monobeast import batch, unbatch
from nmmo import Env
def torchify_features(features: dict) -> dict:
    out_f = {}
    for ag_id, ag_dict in features.items():
        ag_out_dict = {}
        for f_key, f_val in ag_dict.items():
            ag_out_dict[f_key] = torch.from_numpy(f_val).unsqueeze(0).unsqueeze(0) # to add agent dimension and team dimension
        out_f[ag_id] = ag_out_dict
    return out_f

def stack_data(input: dict) -> torch.Tensor:
    assert "va" in input
    terrain, camp, entity = input["terrain"], input[
                "camp"], input["entity"]
    terrain = F.one_hot(terrain, num_classes=6).permute(0, 1, 4, 2, 3)
    camp = F.one_hot(camp, num_classes=4).permute(0, 1, 4, 2, 3)

    # print(terrain.shape, camp.shape, entity.shape)
    x = torch.cat([terrain, camp, entity], dim=2)
    return x


/home/penzard/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# Default environment - see API for config options
env = Env()
obs = env.reset()


actions = {} # Compute with your model


obs, rewards, dones, infos = env.step(actions)
parser = FeatureParser()
features = torchify_features(parser.parse(obs))
batched_features, agent_ids = batch(features, filter_keys=list(parser.feature_spec))



inp = stack_data(batched_features)

/home/penzard/miniconda3/envs/nmmoenv2/lib/python3.8/site-packages/nmmo/core/terrain.py:53: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  tex = imread(path.format(key))


In [4]:
import torch.nn as nn
cnn = nn.Sequential(
            nn.Conv2d(in_channels=17,
                      out_channels=32,
                      kernel_size=3,
                      stride=1,
                      padding=1), nn.ReLU(),
            nn.Conv2d(in_channels=32,
                      out_channels=32,
                      kernel_size=3,
                      stride=1,
                      padding=1), nn.ReLU(),
            nn.Conv2d(in_channels=32,
                      out_channels=32,
                      kernel_size=3,
                      stride=1,
                      padding=1), nn.ReLU(),
            nn.Conv2d(in_channels=32,
                      out_channels=16,
                      kernel_size=3,
                      stride=1,
                      padding=1), nn.ReLU(),
            nn.Conv2d(in_channels=16,
                      out_channels=16,
                      kernel_size=3,
                      stride=1,
                      padding=1), nn.ReLU())

In [5]:
joined_input = torch.flatten(inp, 0, 1)


cnn(joined_input).shape

torch.Size([4, 16, 15, 15])